In [11]:
import os
import re
import datetime
import pickle as pkl
import pandas as pd
import requests

This requires the full HTML files of the PR from GitHub.  
(Full meaning that the whole timeline needs to be expanded by clicking 'Load more...' where ever possible.)

We only need the reviews and the review comments. Normal comments are returned by the API.

In [12]:
folder_path = "pr_html"

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    
    match = re.search(r"#(\d+)", filename)
    if match:
        number = match.group(1)
        
        new_filename = f"{number}.html"
        new_file_path = os.path.join(folder_path, new_filename)
        os.rename(file_path, new_file_path)

In [13]:
folder_path = "pr_html"

r_reviews = r'diff-for-comment-(\d*).*?github.com\/(.*?)">\2<\/a>.*?#discussion-diff-\1.*?datetime="(.*?)"'
r_review_comments = r'review-comment.*?github.com\/(.*?)">\1<\/a>.*?discussion_r(\d*)-permalink.*?datetime="(.*?)"'

data = {}

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    with open(file_path, "r") as file:
        reviews = []
        review_comments = []

        content = file.read()

        review_matches = re.findall(r_reviews, content, re.DOTALL)
        for match in review_matches:
            reviews.append({
                "id": match[0],
                "username": match[1],
                "created_at": datetime.datetime.strptime(match[2], "%Y-%m-%dT%H:%M:%SZ")
            })

        review_comment_matches = re.findall(r_review_comments, content, re.DOTALL)
        for match in review_comment_matches:

            review_comments.append({
                "username": match[0],
                "review_id": match[1],
                "created_at": datetime.datetime.strptime(match[2], "%Y-%m-%dT%H:%M:%SZ")
            })

        data[filename.split(".")[0]] = {
            "reviews": reviews,
            "review_comments": review_comments
        }

pkl.dump(data, open("pr_helper_data.pkl", "wb"))

In [14]:
data

{'12694': {'reviews': [{'id': '36418499',
    'username': 'cbuescher',
    'created_at': datetime.datetime(2015, 8, 6, 14, 13, 46)}],
  'review_comments': [{'username': 'cbuescher',
    'review_id': '36418499',
    'created_at': datetime.datetime(2015, 8, 6, 14, 13, 46)},
   {'username': 'colings86',
    'review_id': '36421322',
    'created_at': datetime.datetime(2015, 8, 6, 14, 36, 15)},
   {'username': 'javanna',
    'review_id': '36500437',
    'created_at': datetime.datetime(2015, 8, 7, 8, 39, 57)}]},
 '7036': {'reviews': [{'id': '15450843',
    'username': 'jpountz',
    'created_at': datetime.datetime(2014, 7, 28, 8, 18, 46)},
   {'id': '15534111',
    'username': 'jpountz',
    'created_at': datetime.datetime(2014, 7, 29, 16, 21, 2)}],
  'review_comments': [{'username': 'jpountz',
    'review_id': '15450843',
    'created_at': datetime.datetime(2014, 7, 28, 8, 18, 46)},
   {'username': 'dadoonet',
    'review_id': '15463271',
    'created_at': datetime.datetime(2014, 7, 28, 14,

In [15]:
data['13788']

{'reviews': [], 'review_comments': []}